# Running Inference on Legacy (Non-Workbench) Models
This notebook goes through the steps of setting up and running inference on Models that weren't generated in Workbench.

```
model = Model("the old model")
model.onboard()
```
Answer all the Model onboarding questions, the tricky one is 'features' (note: capitalization matters)

```
end = Endpoint("the old endpoint")
end.onboard()
```

Answer all the onboard questions, if you're asked for input put the Model Name.

After this is done, you'll need follow this notebook for 'remapping' the evaluation data into a form that the old model can use.

In [ ]:
import os

# Set the environment variable
os.environ['WORKBENCH_CONFIG'] = '/Users/briford/.workbench/scp_sandbox.json'

In [ ]:
# Grab the features from the model (this will be set during onboard())
from workbench.api.model import Model
model = Model("LogS-pH7-Class-0-231025")
target = model.target()
print(target)
features = model.features()
print(f"{features[:5]}...")

In [ ]:
# Grab the evaluation data
from workbench.api.feature_set import FeatureSet
fs = FeatureSet("solubility_featurized_fs")
training_view = fs.view("training").table
eval_data = fs.query(f"select * from {training_view} where training=0")

In [ ]:
data_columns = eval_data.columns

In [ ]:
def build_map(data_columns, features):
    map = dict()
    for f in features:
        if f.lower() in data_columns:
            map[f.lower()] = f
    return map

In [ ]:
map = build_map(data_columns, features)
map["class"] = target  # Add the target variable name change

In [ ]:
# Rename the feature and target columns to 'match' what the Legacy Model expects
eval_data.rename(columns=map, inplace=True)

In [ ]:
# Grab the Legacy Endpoint
from workbench.api.endpoint import Endpoint
end = Endpoint("LogS-pH7-Class-0-231025")

In [ ]:
# Run inference on the Legacy Endpoint
end.inference(eval_data, capture_uuid="2024_03_19_holdout")

In [ ]:
# This might not be strictly necssary but it will make sure the details are up to date
model.details(recompute=True)